In [ ]:
import ReadRawMHD as rR
import test_delete_ as test
import numpy as np
import matplotlib.pyplot as plt
import time
t1 = time.time()

KeyboardInterrupt: 

In [ ]:
# # # # # Input # # # # #
# Input imaging data
sample = 'Pilot3'  # sample folder
bone = 'XCT_Icotec_S130672_L5_intact_planned.mhd'  # gray value bone ct scan
inst = 'ICOTEC_S130672_L5_implants_XCTres.mhd'  # segmented screw scan

In [ ]:
# # # # # Input # # # # #
# Input FEA data



In [ ]:
# Specify file locations
loc_ct = '/home/biomech/Documents/01_Icotec/01_Experiments/02_Scans/' + sample + '/04_Registered/'
loc_fea = 'home/biomech/Documents/01_Icotec/02_FEA/99_Tests/' + sample
info = sample + '.txt'  # .txt file containing info about landmarks. Specific structure

In [ ]:
# Load data
im0 = rR.load_itk(loc_ct + bone)
imD = rR.load_itk(loc_ct + inst)
with open(loc_ct + info, 'r') as f:
    content = f.read()
ori = content.split('Origin: ')[1].split('\n')[0]
ori = np.array([int(ori.split(' ')[0]), int(ori.split(' ')[1]), int(ori.split(' ')[2])])
p1P = content.split('positive x axis: ')[1].split('\n')[0]
p1P = np.array([int(p1P.split(' ')[0]), int(p1P.split(' ')[1]), int(p1P.split(' ')[2])])
p1N = content.split('negative x axis: ')[1].split('\n')[0]
p1N = np.array([int(p1N.split(' ')[0]), int(p1N.split(' ')[1]), int(p1N.split(' ')[2])])

In [ ]:
# # # # # Input # # # # #
# Define screw vector
lineT = rR.axis3D(imD[0], 670, 1100, 'x')  # if starting from screw tip towards head, add (-) in next line
v3 = -lineT.vector  # z-axis = screw axis, found by function. Specify +/-!!

In [ ]:
# Compute matrix
v2 = np.cross(p1P-p1N, v3)/np.linalg.norm(np.cross(p1P-p1N, v3))  # y-axis
v1 = np.cross(v2, v3)  # x-axis
M = np.vstack((np.append(v1, ori[0]), np.append(v2, ori[1]), np.append(v3, ori[2]), np.array([0, 0, 0, 1])))
print(M)

In [ ]:
# Check coordinate system
# z-axis (blue) should point from screw head (origin) towards screw tip (dot)
# y-axis should be perpendicular to screw-rotAxis plane and point upwards (against loading direction)
# x-axis: right-handed coordinate system
plt.figure()
ax = plt.axes(projection='3d')
fact = 100
for i in range(-0, 5):
    ax.scatter3D(ori[0]+i*fact*v1[0], ori[1]+i*fact*v1[1], ori[2]+i*fact*v1[2], c='r', alpha=1)
    ax.scatter3D(ori[0]+i*fact*v2[0], ori[1]+i*fact*v2[1], ori[2]+i*fact*v2[2], c='g', alpha=1)
    ax.scatter3D(ori[0]+i*fact*v3[0], ori[1]+i*fact*v3[1], ori[2]+i*fact*v3[2], c='b', alpha=1)
ax.scatter3D(p1P[0], p1P[1], p1P[2], c='b')
ax.scatter3D([0, 1500], [0, 1500], [0, 1500], alpha=0)
ax.scatter3D(686, 480, 177, c='k', alpha=1)  # Ti tip
plt.show()

In [ ]:
# to do: proceed with fea part here. create separate script with test_delete functions

In [ ]:
print('Execution time: ' + str(int((time.time()-t1)/60)) + ' min '+ str(round(np.mod(time.time()-t1, 60), 1)) + ' sec.')